In [84]:

import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [85]:
n_features = 4
dev = qml.device("default.qutrit", wires=n_features)

In [86]:
# State preparation with angle embedding
def state_prep(x):
    for i, angle in enumerate(x):
        qml.TRX(angle, wires=i, subspace=[0, 1])

In [87]:
def rot(r_weights, wires):
    # Arbitrary rotation on single wire
    qml.TRY(r_weights[0], wires=wires, subspace=[0, 1])
    qml.TRY(r_weights[1], wires=wires, subspace=[0, 2])
    qml.TRY(r_weights[2], wires=wires, subspace=[0, 1])
    qml.TRZ(r_weights[3], wires=wires, subspace=[0, 2])
    qml.TRZ(r_weights[4], wires=wires, subspace=[0, 1])
    qml.TRY(r_weights[5], wires=wires, subspace=[0, 1])
    qml.TRY(r_weights[6], wires=wires, subspace=[0, 2])
    qml.TRY(r_weights[7], wires=wires, subspace=[0, 1])


In [88]:
def layer(l_weights):
    # Trainable layer
    # Arbitrary rotation on all wires
    for i in range(n_features):
        rot(l_weights[i], i)

    # Entanglement gate
    qml.broadcast(unitary=qml.TAdd, pattern="ring", wires=range(n_features))

In [89]:
@qml.qnode(dev)
def classifier(c_weights, x):
    state_prep(x)

    for W in c_weights:
        layer(W)

    return qml.expval(qml.GellMann(wires=0, index=1))

def make_predictions(p_weights, X):
    preds = [classifier(p_weights, x) for x in X]

    for i, pred in enumerate(preds):
        if pred >= 1 / 3:
            preds[i] = 1
        elif pred < -1 / 3:
            preds[i] = -1
        else:
            preds[i] = 0

    return preds

In [111]:
def cost(co_weights):
    preds = [classifier(co_weights, x) for x in X_train]
    loss = 0
    for y, p in zip(Y_train, preds):
        loss += (y - p) ** 2

    return loss / len(Y)

def accuracy(Y, preds):
    acc = 0
    for y, p in zip(Y, preds):
        if np.isclose(y, p):
            acc += 1

    return acc / len(Y)

In [112]:
columns = ["f1", "f2", "f3", "f4", "class"]
data = pd.read_csv("./data/iris.data", sep=",", header=None, names=columns)

class_translation = {
    "Iris-setosa": -1,
    "Iris-versicolor": 0,
    "Iris-virginica": 1,
}

data["class"] = data["class"].replace(class_translation.keys(), class_translation.values())
data = data.to_numpy(dtype=np.float64)


In [113]:
X = np.array(data[:, :-1], requires_grad=False)
Y = np.array(data[:, -1], requires_grad=False)

In [114]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
X_train = X[index[:num_train]]
Y_train = Y[index[:num_train]]
X_val = X[index[num_train:]]
Y_val = Y[index[num_train:]]

In [115]:
opt = qml.GradientDescentOptimizer(stepsize=0.1)

In [116]:
n_layers = 1
weights_init = 0.01 * np.random.randn(n_layers, n_features, 8, requires_grad=True)

In [117]:
n_its = 100
weights = np.array(weights_init, requires_grad=True)

loss_track = []

for it in range(n_its):
    weights, _loss = opt.step_and_cost(cost, weights)
    if it % 5 == 0:
        preds = make_predictions(weights, X_train)
        print(f"Loss at iteration {it:2} = {_loss: .10f}  Accuracy = {accuracy(Y_train, preds): .2f}")
    loss_track.append(_loss)

Loss at iteration  0 =  0.5105393072  Accuracy =  0.32
Loss at iteration  5 =  0.5101810828  Accuracy =  0.32
Loss at iteration 10 =  0.5098067115  Accuracy =  0.32
Loss at iteration 15 =  0.5094083412  Accuracy =  0.32
Loss at iteration 20 =  0.5089763999  Accuracy =  0.32
Loss at iteration 25 =  0.5084989032  Accuracy =  0.32
Loss at iteration 30 =  0.5079606104  Accuracy =  0.32
Loss at iteration 35 =  0.5073420003  Accuracy =  0.32
Loss at iteration 40 =  0.5066180605  Accuracy =  0.32
Loss at iteration 45 =  0.5057569233  Accuracy =  0.32
Loss at iteration 50 =  0.5047184646  Accuracy =  0.32
Loss at iteration 55 =  0.5034531227  Accuracy =  0.32
Loss at iteration 60 =  0.5019014112  Accuracy =  0.32
Loss at iteration 65 =  0.4999948741  Accuracy =  0.32
Loss at iteration 70 =  0.4976594905  Accuracy =  0.32
Loss at iteration 75 =  0.4948225450  Accuracy =  0.32
Loss at iteration 80 =  0.4914233907  Accuracy =  0.32
Loss at iteration 85 =  0.4874269758  Accuracy =  0.32
Loss at it